In [1]:
%run ../../utils/commonImports.py
%run ../../utils/tradingImports.py
%matplotlib inline

from base import supres, BaseStrategy
from exputils import *

# Load Data

In [2]:
# dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'

In [3]:
pairs = pd.read_csv('selected-pairs.tsv', header=None)[0].tolist()
train_dict = {pair: read_train(pair, dropbox_dir) for pair in pairs}

# Experiment setup

In [23]:
from itertools import product


features = ['support']
window_sizes = [4, 6, 12]

targets = []
for t1 in range(1, 11):
    for t2 in range(t1+2, 11):
        targets.append([t1/100, t2/100])

stop_losses = list(np.linspace(0, 0.1, 11))[1:]
returns = [i / 100 for i in range(1, 6)]

args = product(pairs, window_sizes, targets, stop_losses, returns)
args = filter(lambda x: x[3] <= 2 * x[2][-1], args)
args = list(args)
len(args)

100320

# Try params

In [25]:
supports_for_data_dict = load_model('ohlc_supports_train.pkl')

In [26]:
def trading_stats(data, pair, profit_targets, willing_loss, _return):
    contract_pair = ContractPair.new(*[x.lower() for x in pair.split('_')])
    dm = OhlcDataManager(data[['open', 'low', 'high', 'close']], data[features])
    wallet = {'btc': 100}
    exchange = BackTestExchange(dm, wallet, 0.0025)
    strategy = BaseStrategy(exchange, 
                            dm, 
                            contract_pair, 
                            willing_loss=willing_loss,  
                            target_return=_return,
                            profit_targets=profit_targets)
    strategy.trade()
    strategy_stats = strategy.stats(contract_pair['priceContract']).report()
    return strategy_stats


def trading_stats_f(args):
    pair, window_size, profit_targets, stop_loss, _return = args
    
    data = train_dict[pair]
    data['support'] = supports_for_data_dict[pair][window_size]
    stats = trading_stats(data, pair, profit_targets, stop_loss, _return)
    return pair, window_size, profit_targets, stop_loss, _return, stats

In [27]:
results = run_parallel(trading_stats_f, args)

param_cols = ['pair', 'window_size', 'profit_targets', 'stop_loss', '_return']
stats_cols = ['startAmount', 'numberOfTrades', 'totalProfit', 'avgProfit',
    'winPercentage', 'avgWinTrade', 'avgLossTrade', 'profitFactor',
    'maxDrawdown', 'avgTimeToClose', 'minTimeToClose', 'maxTimeToClose']

result_df = pd.DataFrame(results, columns=param_cols+['stats'])

for c in stats_cols:
    result_df[c] = result_df['stats'].apply(lambda x: x.iloc[:, 0][c])
    
result_df = result_df.drop('stats', axis=1)

rpath = os.path.join(dropbox_dir,'result', 'base-strategy2-learn-01.pkl')
result_df.to_pickle(rpath)